In [25]:
import pandas as pd
import psycopg2
import config
import sys

sys.path.append("..")

import datetime
import requests, zipfile
import io
from setts import regex
import workdays
from functools import reduce
holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')


dt = datetime.date.today()
dt_1 = workdays.workday(dt, -1, holidays_b3)
dt_2 = workdays.workday(dt, -2, holidays_b3)
dt_2 = workdays.workday(dt, -2, holidays_b3)
dt = datetime.date.today()
dt_1 = workdays.workday(dt, -1, holidays_b3)



In [26]:
file= f"G:\Controle e Risco\Controle\Previas\Mesas\K11\Gerencial\Gerencial_Fundos_{dt_1.strftime('%Y%m%d')}_Kapitalo 11.1.xlsx"

In [69]:
df

In [27]:
funds=['KappaMaster','KappaPrev','K10Prev','K10Master']
types = ['BPS','Fin']

In [28]:
#
dfs = {
    'Fin': {},
    'BPS': {}
}
bps = []
fin = []
for fund in funds:
    for x in types:
        df=pd.read_excel(file,sheet_name=f'{fund}|{x}')
        
        df=df.loc[4:]
#         display(df)
        df=df[['Unnamed: 35','Unnamed: 42']].reset_index(drop=True)
        df.columns=df.iloc[0]
        df=df.drop(0).fillna(0)
        dfs[x][fund] = df[df["Kapitalo 11.1"]!=0].rename(columns={'Kapitalo 11.1':fund})
        if x == 'BPS':
            bps.append(dfs[x][fund])
        else:
            fin.append(dfs[x][fund])
df_bps = reduce(lambda left,right: pd.merge(left,right,on='Estratégia Mesa Dia',how='left'),bps,)
df_fin = reduce(lambda left,right: pd.merge(left,right,on='Estratégia Mesa Dia',how='left'),fin,)

In [29]:
strat = dict()

In [34]:
for i,fund in enumerate(funds):
    if fund!='K10Master':
        for strategy in dfs['Fin'][fund]['Estratégia Mesa Dia'].tolist():
            if strategy!="ROLAGEM_B3":
                if i==0:
                    strat[strategy]=(float(dfs['Fin'][fund][dfs['Fin'][fund]['Estratégia Mesa Dia']==strategy][fund]))  
                else:
                    strat[strategy]+=(float(dfs['Fin'][fund][dfs['Fin'][fund]['Estratégia Mesa Dia']==strategy][fund]))

In [35]:
df_fin=df_fin.fillna(0).set_index('Estratégia Mesa Dia').drop(columns=['K10Master'])


In [36]:
def prop_classB(dt):
    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)

    query = f"""SELECT * \
            FROM public.tbl_proporcao_fundos where dte_data='{dt}' and str_tipo='Com Master 1'
            """
    df=pd.read_sql(query,db_conn_test)
    db_conn_test.close()
    names= {
    'MASTER 1':    'KappaMaster',
    'KAPPA PREV MASTER': 'KappaPrev',
    'K10 PREV MASTER':    'K10Prev',
    }


    df['str_fundo']=df['str_fundo'].map(names)
    
    df=df.head()

    return dict(zip(df.str_fundo,df.dbl_proporcao))


In [37]:
prop=prop_classB(dt_1)

In [38]:
df_fin

,KappaMaster,KappaPrev,K10Prev
Estratégia Mesa Dia,,,
Arbitragem,-2.176446e+06,-68592.160531,-284524.961337
Arbitragem Aluguel,-3.328955e+04,-1303.544904,-2834.508479
Bolsa,1.961826e+05,0.000000,0.000000
Bonds,1.239775e+05,2485.986389,529.850956
Box_3pontas,-1.079200e+04,0.000000,0.000000
Box_4pontas,-1.925255e+04,828.593322,-69.172868
Caixa,4.653242e+05,19239.536212,115239.691502
CashCarry,3.125772e+05,0.000000,0.000000
CashCarry Commodities,1.052191e+05,13103.651213,9206.389971


In [11]:
for strategy in strat:
    for fund in funds:
        if fund != 'K10Master':
            if int(df_fin.loc[strategy,fund]) != 0:
                df_fin.loc[strategy,fund] = 100*(df_fin.loc[strategy,fund] -  strat[strategy]*prop[fund])/df_fin.loc[strategy,fund]


In [39]:
df_fin

,KappaMaster,KappaPrev,K10Prev
Estratégia Mesa Dia,,,
Arbitragem,-2.176446e+06,-68592.160531,-284524.961337
Arbitragem Aluguel,-3.328955e+04,-1303.544904,-2834.508479
Bolsa,1.961826e+05,0.000000,0.000000
Bonds,1.239775e+05,2485.986389,529.850956
Box_3pontas,-1.079200e+04,0.000000,0.000000
Box_4pontas,-1.925255e+04,828.593322,-69.172868
Caixa,4.653242e+05,19239.536212,115239.691502
CashCarry,3.125772e+05,0.000000,0.000000
CashCarry Commodities,1.052191e+05,13103.651213,9206.389971


In [43]:
# df = dfs['Fin']['KappaMaster'].merge(dfs['Fin']['K10Master'],on='Estratégia Mesa Dia',how='left')

In [7]:
dfs['Fin']['KappaMaster']

,Estratégia Mesa Dia,KappaMaster
1,Arbitragem,-661985.283578
2,Bolsa,10355.055497
3,Bonds,10431.502183
4,Box_3pontas,7790.723860
5,Box_4pontas,-58273.674094
6,Caixa,-34288.463236
7,CashCarry,17090.941782
8,CashCarry Commodities,-10811.508488
9,CashCarry2,-38.332513
10,CashCarry5,135297.746749


In [40]:
df_bps.fillna(0).set_index('Estratégia Mesa Dia')

,KappaMaster,KappaPrev,K10Prev,K10Master
Estratégia Mesa Dia,,,,
Arbitragem,-8.989193,-4.069385,-1.926801,-0.836660
Arbitragem Aluguel,-0.135062,-0.050286,-0.018223,-0.012748
Bolsa,0.791247,0.000000,0.000000,0.074427
Bonds,0.478722,0.216405,0.021907,0.045449
Box_3pontas,-0.043616,0.000000,0.000000,-0.004290
Box_4pontas,-0.089142,0.041831,-0.000380,-0.008155
Caixa,1.879817,0.803388,0.799226,1.066415
CashCarry,1.255914,0.000000,0.000000,0.119203
CashCarry Commodities,0.424361,0.550381,0.062668,0.039641
